# Google solution challenge 

In [1]:
%reload_ext sql

# 1.Manual approach to getting data on scholarships 

## Database on Scholarships 

In [2]:
%sql sqlite:///scholarship.db

'Connected: @scholarship.db'

In [3]:
%%sql

DROP TABLE IF EXISTS scholarships;
CREATE TABLE scholarships(
   Country TEXT,
   City TEXT,
   University TEXT,
   Scholarship TEXT,
   Eligibility_requirements TEXT, 
   Type TEXT,
   Amount_per_year TEXT, 
   Application_Deadline TEXT,
   PRIMARY KEY(Scholarship)
 );

 * sqlite:///scholarship.db
Done.
Done.


[]

In [4]:
%%sql

INSERT INTO scholarships VALUES ('UK', 'London', 'London School of Economics and Political Science', 'The Beacon Scholarship at LSE','This scholarship is for an applicant from Kenya, Tanzania, or Uganda. All undergraduate programmes', 'Academic/Need-based', 20000.0, '28 April 2022');
INSERT INTO scholarships VALUES ('UK', 'London', 'London School of Economics and Political Science', 'American Friends of LSE Scholarship','American citizens', 'financial need and academic merit', NULL, '5pm UK time on 27 April 2020');

 * sqlite:///scholarship.db
1 rows affected.
1 rows affected.


[]

In [5]:
%sql SELECT * FROM scholarships database;

 * sqlite:///scholarship.db
Done.


Country,City,University,Scholarship,Eligibility_requirements,Type,Amount_per_year,Application_Deadline
UK,London,London School of Economics and Political Science,The Beacon Scholarship at LSE,"This scholarship is for an applicant from Kenya, Tanzania, or Uganda. All undergraduate programmes",Academic/Need-based,20000.0,28 April 2022
UK,London,London School of Economics and Political Science,American Friends of LSE Scholarship,American citizens,financial need and academic merit,None,5pm UK time on 27 April 2020


## Database on mentors

In [6]:
%sql sqlite:///mentors.db

'Connected: @mentors.db'

In [7]:
%%sql

DROP TABLE IF EXISTS mentors;
CREATE TABLE mentors(
   Sirname TEXT,
   Name TEXT,
   Country TEXT,
   City TEXT,
   University TEXT,
   Course TEXT,
   Availability INT,
   PRIMARY KEY(Sirname)
 );

 * sqlite:///mentors.db
   sqlite:///scholarship.db
Done.
Done.


[]

In [8]:
%%sql

INSERT INTO mentors VALUES ('Sugarman', 'Ben', 'UK', 'London', 'London School of Economics and Political Science', 'Data Science', 7);

 * sqlite:///mentors.db
   sqlite:///scholarship.db
1 rows affected.


[]

In [9]:
%sql SELECT * FROM mentors database;

 * sqlite:///mentors.db
   sqlite:///scholarship.db
Done.


Sirname,Name,Country,City,University,Course,Availability
Sugarman,Ben,UK,London,London School of Economics and Political Science,Data Science,7


# Using web scrabbing to autonomise data collection on available scholarships 

In [10]:
from bs4 import BeautifulSoup
import requests
import re
url = 'https://www.lse.ac.uk/study-at-lse/Undergraduate/fees-and-funding/LSE-financial-support-Overseas-students'
r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')

str_data=[]
data= soup.find_all('p') 
for i in data:
    str_data.append(i.text)

scholarship_regex_results=[]

for i in str_data:
    scholarship_regex_results.append(re.search("Scholarship| Support Scheme",i)) #we could add way more key words

count=-1        
for i in scholarship_regex_results:
    count+=1
    if i != None:
        print(str_data[count])
    else:
        pass

LSE Undergraduate Scholarship Application form
The deadline for the receipt of LSE Undergraduate Scholarship Applications for offer holders with Overseas fee status is: 28 April 2022
By submitting your completed LSE Undergraduate Scholarship Application, you will be considered for any LSE scholarships for which you are eligible. Scholarships that are to be awarded to new students joining LSE in the 2022/23 academic year:  
The Beacon Scholarship at LSE
ICAP Undergraduate Scholarship
LSE Undergraduate Support Scheme
Pestalozzi International Village Trust Scholarships
LSE Mauritius Scholarship
LSE Stelios Scholarship
Stonex Undergraduate Scholarship for EU Students
Uggla Family Scholarships


In [11]:
import pandas as pd 
df=pd.read_html('https://www.lse.ac.uk/study-at-lse/Undergraduate/fees-and-funding/LSE-financial-support-Overseas-students')
df=df[0]
df

,Scholarship name,Scholarship value,Number of scholarships,Eligibility
0,The Beacon Scholarship at LSE,Combined support from LSE and The Beacon Trust,one,"Student from Kenya, Tanzania or Uganda studyin..."
1,ICAP Undergraduate Scholarship,TBC,one,Student from Sub-Saharan Africa studying on an...
2,LSE Undergraduate Support Scheme,varies depending on individual circumstances,mutiple,Overseas fee status students studying any prog...
3,Pestalozzi International Village Trust Scholar...,The cost of full fees and living costs,two,Overseas students sponsored by the Pestalozzi ...
4,LSE Mauritius Scholarship,"£23,330 per year",one,Mauritian student studying on any programme
5,LSE Stelios Scholarship,"£10,000 per year",three,Greek and Cypriot students studying within the...
6,Stonex Undergraduate Scholarship for EU Students,"£10,000 per year",one,Students from the EU studying on any programme
7,Uggla Family Scholarships,"£27,526 per year for fees and living costs",three,Overseas fee status students studying any prog...


## Data manipulation on pandas dataframe 

In [49]:
scholarship=[]
eligibility_requirements=[]
Type=[]
amount_per_year=[]
application_Deadline=[]
for row in df.iterrows():
    
    scholarship.append(row[1][0])
    
    amount_per_year.append(row[1][1])
    
    eligibility_requirements.append(row[1][3])

In [13]:
pandas_database_version_lse = pd.DataFrame(list(zip(scholarship,amount_per_year,eligibility_requirements)), 
                                           columns =['Scholarship','Amount_per_year','Eligibility_requirements'])

pandas_database_version_lse['Country']='UK'
pandas_database_version_lse['City']='London'
pandas_database_version_lse['University']='London School of Economics and Political Science'

pandas_database_version_lse['type']='Missing information'
pandas_database_version_lse['Application_Deadline']='Missing information'
pandas_database_version_lse

,Scholarship,Amount_per_year,Eligibility_requirements,Country,City,University,type,Application_Deadline
0,The Beacon Scholarship at LSE,Combined support from LSE and The Beacon Trust,"Student from Kenya, Tanzania or Uganda studyin...",UK,London,London School of Economics and Political Science,Missing information,Missing information
1,ICAP Undergraduate Scholarship,TBC,Student from Sub-Saharan Africa studying on an...,UK,London,London School of Economics and Political Science,Missing information,Missing information
2,LSE Undergraduate Support Scheme,varies depending on individual circumstances,Overseas fee status students studying any prog...,UK,London,London School of Economics and Political Science,Missing information,Missing information
3,Pestalozzi International Village Trust Scholar...,The cost of full fees and living costs,Overseas students sponsored by the Pestalozzi ...,UK,London,London School of Economics and Political Science,Missing information,Missing information
4,LSE Mauritius Scholarship,"£23,330 per year",Mauritian student studying on any programme,UK,London,London School of Economics and Political Science,Missing information,Missing information
5,LSE Stelios Scholarship,"£10,000 per year",Greek and Cypriot students studying within the...,UK,London,London School of Economics and Political Science,Missing information,Missing information
6,Stonex Undergraduate Scholarship for EU Students,"£10,000 per year",Students from the EU studying on any programme,UK,London,London School of Economics and Political Science,Missing information,Missing information
7,Uggla Family Scholarships,"£27,526 per year for fees and living costs",Overseas fee status students studying any prog...,UK,London,London School of Economics and Political Science,Missing information,Missing information


## Integrating input simulating users use of a website/database

In [ ]:

print('Select country: ')
country_input=input()
    
print('Select city: ')
city_input=input()

print('Select university: ')
university_input=input()

print('Select the name of scholarship you wish to gain more information on: ')
scholarship_input=input()


Select country: 


In [17]:
def input_function(country_input,city_input,university_input,scholarship_input):
    
    for i in range(len(pandas_database_version_lse)):
        
        if pandas_database_version_lse['Scholarship'][i]==scholarship_input:
            return pandas_database_version_lse[pandas_database_version_lse['Scholarship']==scholarship_input]
        
        elif pandas_database_version_lse['University'][i]==university_input:
            return pandas_database_version_lse[pandas_database_version_lse['University']==university_input]
            
        elif pandas_database_version_lse['City'][i]==city_input:
                                         return pandas_database_version_lse[pandas_database_version_lse['City']==city_input]
            
        elif pandas_database_version_lse['Country'][i]==country_input:
                                         return pandas_database_version_lse[pandas_database_version_lse['Country']==country_input]
input_function(country_input,city_input,university_input,scholarship_input)

,Scholarship,Amount_per_year,Eligibility_requirements,Country,City,University,type,Application_Deadline
0,The Beacon Scholarship at LSE,Combined support from LSE and The Beacon Trust,"Student from Kenya, Tanzania or Uganda studyin...",UK,London,London School of Economics and Political Science,Missing information,Missing information
1,ICAP Undergraduate Scholarship,TBC,Student from Sub-Saharan Africa studying on an...,UK,London,London School of Economics and Political Science,Missing information,Missing information
2,LSE Undergraduate Support Scheme,varies depending on individual circumstances,Overseas fee status students studying any prog...,UK,London,London School of Economics and Political Science,Missing information,Missing information
3,Pestalozzi International Village Trust Scholar...,The cost of full fees and living costs,Overseas students sponsored by the Pestalozzi ...,UK,London,London School of Economics and Political Science,Missing information,Missing information
4,LSE Mauritius Scholarship,"£23,330 per year",Mauritian student studying on any programme,UK,London,London School of Economics and Political Science,Missing information,Missing information
5,LSE Stelios Scholarship,"£10,000 per year",Greek and Cypriot students studying within the...,UK,London,London School of Economics and Political Science,Missing information,Missing information
6,Stonex Undergraduate Scholarship for EU Students,"£10,000 per year",Students from the EU studying on any programme,UK,London,London School of Economics and Political Science,Missing information,Missing information
7,Uggla Family Scholarships,"£27,526 per year for fees and living costs",Overseas fee status students studying any prog...,UK,London,London School of Economics and Political Science,Missing information,Missing information


## filter by Amount_per_year 

In [26]:
full_results=[]

for i in pandas_database_version_lse['Amount_per_year']:
    full_results.append(re.search("full | Full |Total | Complete |Full-ride ",i))

def full_function(full_results):
    count=-1        
    for i in full_results:
        count+=1
        if i != None:
            full_requirement= pandas_database_version_lse['Amount_per_year'][count]
            print(full_requirement)
            return pandas_database_version_lse[pandas_database_version_lse['Amount_per_year']==full_requirement]
        
full_function(full_results)

The cost of full fees and living costs


,Scholarship,Amount_per_year,Eligibility_requirements,Country,City,University,type,Application_Deadline
3,Pestalozzi International Village Trust Scholar...,The cost of full fees and living costs,Overseas students sponsored by the Pestalozzi ...,UK,London,London School of Economics and Political Science,Missing information,Missing information


## Deriving insights on competitiveness of scholarships 

In [48]:
pandas_database_version_lse['Applications']=0
pandas_database_version_lse['Scholarships received']=0
pandas_database_version_lse['Applications']= pandas_database_version_lse['Applications'][3]=8
pandas_database_version_lse['Applications'][4]=20
pandas_database_version_lse['Scholarships received'][3]=4
pandas_database_version_lse['Scholarships received'][4]=4
pandas_database_version_lse['Ratio of successful applications']= pandas_database_version_lse['Scholarships received']/ pandas_database_version_lse['Applications']
pandas_database_version_lse.sort_values(by = 'Ratio of successful applications', ascending=False)

<ipython-input-48-ad07f8ed650a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_database_version_lse['Applications']= pandas_database_version_lse['Applications'][3]=8
<ipython-input-48-ad07f8ed650a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_database_version_lse['Applications'][4]=20
<ipython-input-48-ad07f8ed650a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_database_version_lse['Scholarships received']

,Scholarship,Amount_per_year,Eligibility_requirements,Country,City,University,type,Application_Deadline,Applications,Scholarships received,Ratio of successful applications
3,Pestalozzi International Village Trust Scholar...,The cost of full fees and living costs,Overseas students sponsored by the Pestalozzi ...,UK,London,London School of Economics and Political Science,Missing information,Missing information,8,4,0.5
4,LSE Mauritius Scholarship,"£23,330 per year",Mauritian student studying on any programme,UK,London,London School of Economics and Political Science,Missing information,Missing information,20,4,0.2
0,The Beacon Scholarship at LSE,Combined support from LSE and The Beacon Trust,"Student from Kenya, Tanzania or Uganda studyin...",UK,London,London School of Economics and Political Science,Missing information,Missing information,8,0,0.0
1,ICAP Undergraduate Scholarship,TBC,Student from Sub-Saharan Africa studying on an...,UK,London,London School of Economics and Political Science,Missing information,Missing information,8,0,0.0
2,LSE Undergraduate Support Scheme,varies depending on individual circumstances,Overseas fee status students studying any prog...,UK,London,London School of Economics and Political Science,Missing information,Missing information,8,0,0.0
5,LSE Stelios Scholarship,"£10,000 per year",Greek and Cypriot students studying within the...,UK,London,London School of Economics and Political Science,Missing information,Missing information,8,0,0.0
6,Stonex Undergraduate Scholarship for EU Students,"£10,000 per year",Students from the EU studying on any programme,UK,London,London School of Economics and Political Science,Missing information,Missing information,8,0,0.0
7,Uggla Family Scholarships,"£27,526 per year for fees and living costs",Overseas fee status students studying any prog...,UK,London,London School of Economics and Political Science,Missing information,Missing information,8,0,0.0
